# 自定义损失函数
PyTorch在torch.nn模块为我们提供了许多常用的损失函数，比如：MSELoss，L1Loss，BCELoss...... 但是随着深度学习的发展，出现了越来越多的非官方提供的Loss，比如DiceLoss，HuberLoss，SobolevLoss...... 这些Loss Function专门针对一些非通用的模型，PyTorch不能将他们全部添加到库中去，因此这些损失函数的实现则需要我们通过自定义损失函数来实现。另外，在科学研究中，我们往往会提出全新的损失函数来提升模型的表现，这时我们既无法使用PyTorch自带的损失函数，也没有相关的博客供参考，此时自己实现损失函数就显得更为重要了。

经过本节的学习，你将收获：

 - 掌握如何自定义损失函数

## 以函数方式定义
事实上，损失函数仅仅是一个函数而已，因此我们可以通过直接以函数定义的方式定义一个自己的函数，如下所示：

In [1]:
def my_loss(output, target):
    loss = torch.mean((output - target)**2)
    return loss

## 以类方式定义
虽然以函数定义的方式很简单，但是以类方式定义更加常用，在以类方式定义损失函数时，我们如果看每一个损失函数的继承关系我们就可以发现Loss函数部分继承自_loss, 部分继承自_WeightedLoss, 而_WeightedLoss继承自_loss， _loss继承自 nn.Module。我们可以将其当作神经网络的一层来对待，同样地，我们的损失函数类就需要继承自nn.Module类，在下面的例子中我们以DiceLoss为例向大家讲述。

Dice Loss是一种在分割领域常见的损失函数，定义如下：
$$ DSC = \frac{2|X∩Y|}{|X|+|Y|} $$ 
实现代码如下：

In [10]:
import torch.nn as nn
#import torch.nn.functional as F
class DiceLoss(nn.Module):
    def __init__(self,weight=None,size_average=True):
        super(DiceLoss,self).__init__()
        
    def forward(self,inputs,targets,smooth=1):
        #注意torch.nn.functional 已经直接用torch来调用
        inputs = torch.sigmoid(inputs)       
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        intersection = (inputs * targets).sum()                   
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        return 1 - dice

# 使用方法    
criterion = DiceLoss()
#随机生成张量
input = torch.randn(3, requires_grad=True)
target = torch.empty(3).random_(2)
loss = criterion(input,targets)
print(loss)

tensor(0.5900, grad_fn=<RsubBackward1>)


In [6]:
class DiceBCELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        inputs = torch.sigmoid(inputs)       
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        intersection = (inputs * targets).sum()                     
        dice_loss = 1 - (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        BCE = torch.binary_cross_entropy(inputs, targets, reduction='mean')
        Dice_BCE = BCE + dice_loss
        
        return Dice_BCE

    
class IoULoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(IoULoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        inputs = torch.sigmoid(inputs)       
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        intersection = (inputs * targets).sum()
        total = (inputs + targets).sum()
        union = total - intersection 
        
        IoU = (intersection + smooth)/(union + smooth)
                
        return 1 - IoU
    
ALPHA = 0.8
GAMMA = 2

class FocalLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(FocalLoss, self).__init__()

    def forward(self, inputs, targets, alpha=ALPHA, gamma=GAMMA, smooth=1):
        inputs = torch.sigmoid(inputs)       
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        BCE = torch.binary_cross_entropy(inputs, targets, reduction='mean')
        BCE_EXP = torch.exp(-BCE)
        focal_loss = alpha * (1-BCE_EXP)**gamma * BCE
                       
        return focal_loss

# 动态调整学习率
学习率的选择是深度学习中一个困扰人们许久的问题，学习速率设置过小，会极大降低收敛速度，增加训练时间；学习率太大，可能导致参数在最优解两侧来回振荡。但是当我们选定了一个合适的学习率后，经过许多轮的训练后，可能会出现准确率震荡或loss不再下降等情况，说明当前学习率已不能满足模型调优的需求。此时我们就可以通过一个适当的学习率衰减策略来改善这种现象，提高我们的精度。这种设置方式在PyTorch中被称为scheduler，也是我们本节所研究的对象。

本节主要内容：
- 如何根据需要选取已有的学习率调整策略
- 如何自定义设置学习调整策略并实现

## 使用官方scheduler
PyTorch已经在torch.optim.lr_scheduler为我们封装好了一些动态调整学习率的方法供我们使用。
我们在使用官方给出的torch.optim.lr_scheduler时，需要将scheduler.step()放在optimizer.step()后面进行使用。
具体方法请查看https://pytorch.org/docs/stable/optim.html

## 自定义scheduler
虽然PyTorch官方给我们提供了许多的API，但是在实验中也有可能碰到需要我们自己定义学习率调整策略的情况，而我们的方法是自定义函数adjust_learning_rate来改变param_group中lr的值，在下面的叙述中会给出一个简单的实现。

假设我们现在正在做实验，需要学习率每30轮下降为原来的1/10，假设已有的官方API中没有符合我们需求的，那就需要自定义函数来实现学习率的改变。

In [13]:
def adjust_learning_rate(optimizer, epoch):
    lr = args.lr * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

有了adjust_learning_rate函数的定义，在训练的过程就可以调用我们的函数来实现学习率的动态变化

In [16]:
'''
def adjust_learning_rate(optimizer,...):
    ...
optimizer = torch.optim.SGD(model.parameters(),lr = args.lr,momentum = 0.9)
for epoch in range(10):
    train(...)
    validate(...)
    adjust_learning_rate(optimizer,epoch)
'''


def adjust_learning_rate(optimizer):
    optimizer = torch.optim.SGD(model.parameters(),lr = args.lr,momentum = 0.9)
    for epoch in range(10):
        train(...)
        validate(...)
        adjust_learning_rate(optimizer,epoch)

# 模型微调
其中心问题：训练大数据集的模型在训练小数据集上会出现过拟合问题。（大模型的参数多，数据量小会导致模型的参数冗余出现过拟合）

解决办法：
 - 增大数据量，成本会大幅增加
 - 应用迁移学习(transfer learning)，将从源数据集学到的知识迁移到目标数据集上。例如，虽然ImageNet数据集的图像大多跟椅子无关，但在该数据集上训练的模型可以抽取较通用的图像特征，从而能够帮助识别边缘、纹理、形状和物体组成等。这些类似的特征对于识别椅子也可能同样有效。
 
迁移学习的一大应用场景是模型微调（finetune）。简单来说，就是我们先找到一个同类的别人训练好的模型，把别人现成的训练好了的模型拿过来，换成自己的数据，通过训练调整一下参数。 在PyTorch中提供了许多预训练好的网络模型（VGG，ResNet系列，mobilenet系列......），这些模型都是PyTorch官方在相应的大型数据集训练好的。学习如何进行模型微调，可以方便我们快速使用预训练模型完成自己的任务。
 
本节内容：
 - 掌握模型微调的流程
 - 了解PyTorch提供的常用model
 - 掌握如何指定训练模型的部分层

## 模型微调的流程
1、在源数据集(如ImageNet数据集)上预训练一个神经网络模型，即源模型。

2、创建一个新的神经网络模型，即目标模型。它复制了源模型上**除了输出层外**的所有模型设计及其参数。*我们假设这些模型参数包含了源数据集上学习到的知识，且这些知识同样适用于目标数据集。我们还假设源模型的输出层跟源数据集的标签紧密相关，因此在目标模型中不予采用。*

3、为目标模型添加一个输出⼤小为⽬标数据集类别个数的输出层，并随机初始化该层的模型参数。

4、在目标数据集上训练目标模型。我们将从头训练输出层，而其余层的参数都是基于源模型的参数微调得到的。

**对于第2点的理解言外之意就是微调不同的模型，我们对不同网络层的调整是不一样的。比如说一些模型的输出层是softmax，其主要目的就是分类，所以我们会假设输出层与数据标签紧密相关)**

**Q：为什么训练好的模型是具有知识的？**
**A：本质上我们训练一个模型，就是不断模型中的参数，使其损失函数（loss）变得更小，当模型被训练好之后，我们模型的参数就是在数据集中使得模型loss最优的参数。**

**Q：为什么可以进行模型微调？**
**A：模型训练好之后，其在源模型下loss已经最优。更换数据之后，此时模型的参数并不是最优，但是因为源模型是面向源大规模数据集的，或者目标数据集是源数据集的一部分，那么此时源模型的参数是具有一定“知识”的。**

**模型微调的本质在已有的参数跟我们目标模型的参数其实更相近（比随机初始化好），所以模型微调可以帮助我们训练一个新的模型。**

## 使用已有模型结构
这里我们以torchvision中的常见模型为例，列出了如何在图像分类任务中使用PyTorch提供的常见模型结构和参数。对于其他任务和网络结构，使用方式是类似的：

- 实例化网络

In [17]:
import torchvision.models as models
resnet18 = models.resnet18()
# resnet18 = models.resnet18(pretrained=False)  等价于与上面的表达式
alexnet = models.alexnet()
vgg16 = models.vgg16()
squeezenet = models.squeezenet1_0()
densenet = models.densenet161()
inception = models.inception_v3()
googlenet = models.googlenet()
shufflenet = models.shufflenet_v2_x1_0()
mobilenet_v2 = models.mobilenet_v2()
mobilenet_v3_large = models.mobilenet_v3_large()
mobilenet_v3_small = models.mobilenet_v3_small()
resnext50_32x4d = models.resnext50_32x4d()
wide_resnet50_2 = models.wide_resnet50_2()
mnasnet = models.mnasnet1_0()

D:\ProgramData\Anaconda3\envs\pytorch\lib\site-packages\torchvision\models\inception.py:82: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  ' due to scipy/scipy#11299), please set init_weights=True.', FutureWarning)
D:\ProgramData\Anaconda3\envs\pytorch\lib\site-packages\torchvision\models\googlenet.py:79: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  ' due to scipy/scipy#11299), please set init_weights=True.', FutureWarning)


- 传递pretrained参数

通过True或者False来决定是否使用预训练好的权重，在默认状态下pretrained = False，意味着我们不使用预训练得到的权重，当pretrained = True，意味着我们将使用在一些数据集上预训练得到的权重。

In [1]:
import torchvision.models as models
resnet18 = models.resnet18(pretrained=True)
alexnet = models.alexnet(pretrained=True)
squeezenet = models.squeezenet1_0(pretrained=True)
vgg16 = models.vgg16(pretrained=True)
densenet = models.densenet161(pretrained=True)
inception = models.inception_v3(pretrained=True)
googlenet = models.googlenet(pretrained=True)
shufflenet = models.shufflenet_v2_x1_0(pretrained=True)
mobilenet_v2 = models.mobilenet_v2(pretrained=True)
mobilenet_v3_large = models.mobilenet_v3_large(pretrained=True)
mobilenet_v3_small = models.mobilenet_v3_small(pretrained=True)
resnext50_32x4d = models.resnext50_32x4d(pretrained=True)
wide_resnet50_2 = models.wide_resnet50_2(pretrained=True)
mnasnet = models.mnasnet1_0(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to C:\Users\baimu chu/.cache\torch\hub\checkpoints\resnet18-5c106cde.pth


Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to C:\Users\baimu chu/.cache\torch\hub\checkpoints\alexnet-owt-4df8aa71.pth


Downloading: "https://download.pytorch.org/models/squeezenet1_0-a815701f.pth" to C:\Users\baimu chu/.cache\torch\hub\checkpoints\squeezenet1_0-a815701f.pth


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\baimu chu/.cache\torch\hub\checkpoints\vgg16-397923af.pth


Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to C:\Users\baimu chu/.cache\torch\hub\checkpoints\densenet161-8d451a50.pth


...

Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-8738ca79.pth" to C:\Users\baimu chu/.cache\torch\hub\checkpoints\mobilenet_v3_large-8738ca79.pth


Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to C:\Users\baimu chu/.cache\torch\hub\checkpoints\mobilenet_v3_small-047dcff4.pth


Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to C:\Users\baimu chu/.cache\torch\hub\checkpoints\resnext50_32x4d-7cdf4587.pth


Downloading: "https://download.pytorch.org/models/wide_resnet50_2-95faca4d.pth" to C:\Users\baimu chu/.cache\torch\hub\checkpoints\wide_resnet50_2-95faca4d.pth


Downloading: "https://download.pytorch.org/models/mnasnet1.0_top1_73.512-f206786ef8.pth" to C:\Users\baimu chu/.cache\torch\hub\checkpoints\mnasnet1.0_top1_73.512-f206786ef8.pth


# 训练特定层
在默认情况下，参数的属性.requires_grad = True，如果我们从头开始训练或微调不需要注意这里。但如果我们正在提取特征并且只想为新初始化的层计算梯度，其他参数不进行改变。那我们就需要通过设置requires_grad = False来冻结部分层。在PyTorch官方中提供了这样一个例程。

In [2]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

在下面我们仍旧使用resnet18为例的将1000类改为4类，但是仅改变最后一层的模型参数，不改变特征提取的模型参数；注意我们先冻结模型参数的梯度，再对模型输出部分的全连接层进行修改，这样修改后的全连接层的参数就是可计算梯度的。`b

In [4]:
import torchvision.models as models
import torch.nn as nn
# 冻结参数的梯度
feature_extract = True
model = models.resnet18(pretrained=True)
set_parameter_requires_grad(model, feature_extract)
# 修改模型
num_ftrs = model.fc.in_features
model.fc = nn.Linear(in_features=512, out_features=4, bias=True)

之后在训练过程中，model仍会进行梯度回传，但是参数更新则只会发生在fc层。通过设定参数的requires_grad属性，我们完成了指定训练模型的特定层的目标，这对实现模型微调非常重要。
